In [1]:
import numpy as np
import pandas as pd
import os
import re
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

In [2]:
story_dir = "/kaggle/input/sherlock-holmes-stories/sherlock"
txt=[]
for _, _, files in os.walk(story_dir):
    for file in files:
        with open(story_dir+"/"+file) as f:
            for line in f:
                line=line.strip()
                if line != '':
                    txt.append(line)


In [3]:
cleaned_txt = []
for line in txt:
    line = line.lower()
    line = re.sub(r"[,.\"\!@#$%^&*(){}?/;`~<>+-\\]","", line)
    tokens = word_tokenize(line)
    words = [word for word in tokens if word.isalpha()]
    cleaned_txt += words

# Markov model

In [4]:
def make_markov_model(cleaned_txt, ngram=2):
    markov_model = {}
    for j in range(len(cleaned_txt)-ngram-1):
        current_word = " ".join(cleaned_txt[j:j+ngram])
        next_word = " ".join(cleaned_txt[j+ngram:j+ngram+ngram])
        if current_word not in markov_model:
            markov_model[current_word] = {}
            markov_model[current_word][next_word] = 1
        else:
            if next_word in markov_model[current_word]:
                markov_model[current_word][next_word] +=1
            else:
                markov_model[current_word][next_word] = 1

    #calculate transition prob
    for curr, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr][state] = count/total

    return markov_model
            
    

In [5]:
markov_model = make_markov_model(cleaned_txt)

In [6]:
def generate_story(markov_model, seq_len=100, start="my dear"):
    story = ""
    story += start
    curr = start
    while seq_len > 0:
        next_set = np.random.choice(list(markov_model[curr].keys()), p=list(markov_model[curr].values()))
        curr = next_set
        story += " " + curr 
        seq_len-=1
    return story

In [7]:
for i in range(10):
    print(str(i) + ": " +generate_story(markov_model, seq_len=4, start="dear holmes"))

0: dear holmes it is indeed a fearful place the torrent
1: dear holmes i have had an unlimited number of stanch
2: dear holmes i fear sir said holmes laughing i think
3: dear holmes said i possibly not so well but clearly
4: dear holmes i have told you that i have not
5: dear holmes i thought that were the initials were of
6: dear holmes that i should not solve the mystery we
7: dear holmes what do you say how could you know
8: dear holmes if i am not aware that he is
9: dear holmes if i gave you a goodly volume under
